In [1]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("..")

In [2]:
from llama_python.configuration import LlamaConfig
from llama_python.model import LlamaForCausalLM, LlamaModel
import torch

In [3]:
import json

config_content: dict = json.load(open("../resources/original_llama/config.json", "r"))
model_config = LlamaConfig(**config_content)

In [4]:
model = LlamaForCausalLM(model_config)

partial_rotary_factor 1.0
inv_freq tensor([1.0000e+00, 6.6360e-01, 4.4037e-01, 2.9223e-01, 1.9392e-01, 1.2869e-01,
        8.5397e-02, 5.6670e-02, 3.7606e-02, 2.4955e-02, 1.6560e-02, 1.0990e-02,
        7.2927e-03, 4.8394e-03, 3.2114e-03, 2.1311e-03, 1.4142e-03, 9.3847e-04,
        6.2277e-04, 4.1327e-04, 2.7425e-04, 1.8199e-04, 1.2077e-04, 8.0143e-05,
        5.3183e-05, 3.5292e-05, 2.3420e-05, 1.5542e-05, 1.0313e-05, 6.8440e-06,
        4.5417e-06, 3.0139e-06])
inv_freq shape torch.Size([32])


In [5]:
from safetensors.torch import load_file
import torch

state_dict = load_file("../model2.safetensors")

list(state_dict.keys())

In [6]:
if "lm_head.weight" not in state_dict:
    model.lm_head.weight = model.get_input_embeddings().weight

In [7]:
missing_keys, unexpected_keys = model.load_state_dict(state_dict, strict=False)
print("Missing keys:", missing_keys)
print("Unexpected keys:", unexpected_keys)

Missing keys: ['lm_head.weight']
Unexpected keys: []


In [8]:
token_ids = [2933, 73, 283, 6634, 91443, 7930, 296, 378, 247, 263, 83]
token_ids = torch.Tensor(token_ids).type(torch.int32).reshape((1, -1))
token_ids, token_ids.shape

(tensor([[ 2933,    73,   283,  6634, 91443,  7930,   296,   378,   247,   263,
             83]], dtype=torch.int32),
 torch.Size([1, 11]))

### Experiments

In [14]:
A = torch.Tensor([
    [1, 2, 3],
    [1, 2, 3],
    [1, 2, 3],
    [1, 2, 3],
    [1, 2, 3],
])
torch.cat((A, A), dim=1).cos()

tensor([[ 0.5403, -0.4161, -0.9900,  0.5403, -0.4161, -0.9900],
        [ 0.5403, -0.4161, -0.9900,  0.5403, -0.4161, -0.9900],
        [ 0.5403, -0.4161, -0.9900,  0.5403, -0.4161, -0.9900],
        [ 0.5403, -0.4161, -0.9900,  0.5403, -0.4161, -0.9900],
        [ 0.5403, -0.4161, -0.9900,  0.5403, -0.4161, -0.9900]])

In [36]:
import torch
from torch import nn, Tensor

class Test(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(in_features=3, out_features=4, bias=False)

    def forward(self, input):
        res = self.layer(input)
        return res


model = Test()
state = {
    "layer.weight": Tensor([
        [1, 2, 3],
        [10, 20, 30],
        [100, 200, 300],
        [1000, 2000, 3000],
    ])
}
model = Test()
model.load_state_dict(state)
print(model.state_dict())

input_data = Tensor([
    [1, 1, 1],
    [2, 2, 2],
])



print(input_data.shape)
with torch.no_grad():
    res = model(input_data)
    print(res, res.shape)

OrderedDict({'layer.weight': tensor([[1.0000e+00, 2.0000e+00, 3.0000e+00],
        [1.0000e+01, 2.0000e+01, 3.0000e+01],
        [1.0000e+02, 2.0000e+02, 3.0000e+02],
        [1.0000e+03, 2.0000e+03, 3.0000e+03]])})
torch.Size([2, 3])
tensor([[6.0000e+00, 6.0000e+01, 6.0000e+02, 6.0000e+03],
        [1.2000e+01, 1.2000e+02, 1.2000e+03, 1.2000e+04]]) torch.Size([2, 4])


In [55]:
data = torch.arange(60).reshape((6, 10))
#data = torch.broadcast_to(data, (6, 10))
data, data.view((6, -1, 5)), data.view((6, -1, 5)).transpose(0, 1)

(tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
         [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
         [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
         [50, 51, 52, 53, 54, 55, 56, 57, 58, 59]]),
 tensor([[[ 0,  1,  2,  3,  4],
          [ 5,  6,  7,  8,  9]],
 
         [[10, 11, 12, 13, 14],
          [15, 16, 17, 18, 19]],
 
         [[20, 21, 22, 23, 24],
          [25, 26, 27, 28, 29]],
 
         [[30, 31, 32, 33, 34],
          [35, 36, 37, 38, 39]],
 
         [[40, 41, 42, 43, 44],
          [45, 46, 47, 48, 49]],
 
         [[50, 51, 52, 53, 54],
          [55, 56, 57, 58, 59]]]),
 tensor([[[ 0,  1,  2,  3,  4],
          [10, 11, 12, 13, 14],
          [20, 21, 22, 23, 24],
          [30, 31, 32, 33, 34],
          [40, 41, 42, 43, 44],
          [50, 51, 52, 53, 54]],
 
         [[ 5,  6,  7,  8,  9],
          [15, 16, 17, 18, 19],
          [25, 26, 27, 28, 29],

In [38]:
data = Tensor([
    [1, 2, 3, 4, 5, 6],
    [10, 20, 30, 40, 50, 60],
    [100, 200, 300, 400, 500, 600],
])
hidden_shape = (3, -1, 2)
data.view(hidden_shape)

tensor([[[  1.,   2.],
         [  3.,   4.],
         [  5.,   6.]],

        [[ 10.,  20.],
         [ 30.,  40.],
         [ 50.,  60.]],

        [[100., 200.],
         [300., 400.],
         [500., 600.]]])

In [73]:
(torch.arange(6) * 2).tolist()

[0, 2, 4, 6, 8, 10]

In [77]:
# query_states torch.Size([1, 32, 11, 64])

x = Tensor([
    [
        [
            (torch.arange(6) * 1).tolist(),
            (torch.arange(6) * 2).tolist(),
            (torch.arange(6) * 3).tolist(),
            (torch.arange(6) * 4).tolist(),
        ]
    ],
    [
        [
            (torch.arange(6) * 10).tolist(),
            (torch.arange(6) * 20).tolist(),
            (torch.arange(6) * 30).tolist(),
            (torch.arange(6) * 40).tolist(),
        ]
        
    ],
    [
        [
            (torch.arange(6) * 100).tolist(),
            (torch.arange(6) * 200).tolist(),
            (torch.arange(6) * 300).tolist(),
            (torch.arange(6) * 400).tolist(),
        ]
        
    ]
])
print(x.shape)
y = x[...,  x.shape[-1]//2 :]
#y = x[..., : x.shape[-1]//2]
y = x[x.shape[-1]//2 :, ...]kkk
print(y.shape)
y

torch.Size([3, 1, 4, 6])
torch.Size([0, 1, 4, 6])


tensor([], size=(0, 1, 4, 6))

### Run model

In [12]:
model(token_ids)

======== LlamaModel.forward =========
input_ids shape= torch.Size([1, 11])
output_attentions False
output_hidden_states False
use_cache True
return_dict True
not in training mode
inputs_embeds torch.Size([1, 11, 2048])
init dynamic cache
init cache_position
past_seen_tokens 0
cache_position shape torch.Size([11])
cache_position tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])
unsequeeze position_ids...
position_ids None
after unsqueeze position_ids shape torch.Size([1, 11])
position_ids tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]])
======== _update_causal_mask =========
attention implementation sdpa
sdpa
>>>>>>>> LlamaModel.forward
======== LlamaRotaryEmbedding.forward =========
rope_type llama3
inv_freq shape torch.Size([32])
inv_freq tensor([1.0000e+00, 6.6360e-01, 4.4037e-01, 2.9223e-01, 1.9392e-01, 1.2869e-01,
        8.5397e-02, 5.6670e-02, 3.7606e-02, 2.4955e-02, 1.6560e-02, 1.0990e-02,
        7.2927e-03, 4.8394e-03, 3.2114e-03, 1.2905e-03, 4.2956e-04, 9.7083e-05,
 

CausalLMOutputWithPast(loss=None, logits=tensor([[[ 5.1046,  5.1268,  7.3774,  ..., -4.7377, -4.7379, -4.7383],
         [11.7675, 12.5821, 10.6723,  ...,  0.8230,  0.8236,  0.8231],
         [13.1911, 12.9374, 10.9242,  ..., -0.3592, -0.3591, -0.3589],
         ...,
         [12.0275, 10.4520, 10.9077,  ...,  0.8572,  0.8575,  0.8566],
         [11.2537,  9.2107,  7.9047,  ..., -1.3219, -1.3206, -1.3209],
         [10.4584,  8.6227,  9.6627,  ..., -1.7254, -1.7234, -1.7235]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=DynamicCache(), hidden_states=None, attentions=None)

In [18]:
res

CausalLMOutputWithPast(loss=None, logits=tensor([[[ 5.1046,  5.1268,  7.3774,  ..., -4.7377, -4.7379, -4.7383],
         [11.7675, 12.5821, 10.6723,  ...,  0.8230,  0.8236,  0.8231],
         [13.1911, 12.9374, 10.9242,  ..., -0.3592, -0.3591, -0.3589],
         ...,
         [12.0275, 10.4520, 10.9077,  ...,  0.8572,  0.8575,  0.8566],
         [11.2537,  9.2107,  7.9047,  ..., -1.3219, -1.3206, -1.3209],
         [10.4584,  8.6227,  9.6627,  ..., -1.7254, -1.7234, -1.7235]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=DynamicCache(), hidden_states=None, attentions=None)